# Multiples Tipos en una Tabla

>Los datasets a menudo involucran valores recopilados en múltiples niveles, en diferentes tipos de unidades de observación. Durante el proceso de *tidying*, cada tipo de unidad de observación debe almacenarse en su propia tabla/dataframe. Esto está estrechamente relacionado con la idea de la normalización de la base de datos, donde cada hecho se expresa en un solo lugar. Si esto no se hace, es posible que ocurran inconsistencias.

## "Housekeeping"

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd

## Ejemplo: Billboard revisited

### Cargando la data
Cargando el dataset limpio y casi "tidy" del notebook 11.

In [3]:
billboard = pd.read_csv("data/billboard_cleaned.csv")

### Messy Data

> El conjunto de datos de Billboard descrito en la Tabla 8 en realidad contiene observaciones sobre dos tipos de
unidades de observación: la **song** y su **rank** en cada semana. Esto se manifiesta a través de la duplicación de "hechos" sobre la canción: `"artist"` y `"time"` se repiten para cada canción en cada `"semana"`.

In [4]:
billboard.head(15)

,year,artist,time,track,date,week,rank
0,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-02-26,1,87
1,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-04,2,82
2,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-11,3,72
3,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-18,4,77
4,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-03-25,5,87
5,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-04-01,6,94
6,2000,2 Pac,4:22,Baby Don't Cry (Keep Ya Head Up II),2000-04-08,7,99
7,2000,2Ge+her,3:15,The Hardest Part Of Breaking Up (Is Getting Ba...,2000-09-02,1,91
8,2000,2Ge+her,3:15,The Hardest Part Of Breaking Up (Is Getting Ba...,2000-09-09,2,87
9,2000,2Ge+her,3:15,The Hardest Part Of Breaking Up (Is Getting Ba...,2000-09-16,3,92


### Tidy Data

> El billboard dataset necesita dividirse en dos conjuntos de datos: un dataframe de **song**, que almacena `"artist"`, `"song name"` y `"time"`, y un conjunto de datos de **rank** que proporciona el `"rank"` de la canción en cada `"week"`.

Transformar columnas de datos en columnas de índice es suficiente en pandas para obtener 'tuplas' únicas de varias columnas. Por lo tanto, no se necesita una "función" real para ordenar el conjunto de datos.

In [5]:
# Get the unique combinations for the song DataFrame and
# "store" them in the original dataset for reuse.
billboard = billboard.set_index(["artist", "track", "time"])

# Create the song DataFrame.
songs = pd.DataFrame.from_records(
    columns=["id", "artist", "track", "time"],
    data=[  # Combine enumerate with tuple unpacking
        (a + 1, b, c, d)  # to create the ID column.
        for (a, (b, c, d)) in enumerate(billboard.index.unique())
    ],
)

# Take the date and rank columns from the original dataset
# and use the implicit index alignment to assign the songs' IDs.
ranking = billboard[["date", "rank"]].copy()
ranking["id"] = songs.set_index(["artist", "track", "time"])

# Use the song ID as the index as in the paper.
ranking = ranking.reset_index(drop=True).set_index("id")
songs = songs.set_index("id")

In [6]:
songs.head(15)

,artist,track,time
id,,,
1,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22
2,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15
3,3 Doors Down,Kryptonite,3:53
4,3 Doors Down,Loser,4:24
5,504 Boyz,Wobble Wobble,3:35
6,98¡,Give Me Just One Night (Una Noche),3:24
7,A*Teens,Dancing Queen,3:44
8,Aaliyah,I Don't Wanna,4:15
9,Aaliyah,Try Again,4:03


In [7]:
ranking.head(15)

,date,rank
id,,
1,2000-02-26,87
1,2000-03-04,82
1,2000-03-11,72
1,2000-03-18,77
1,2000-03-25,87
1,2000-04-01,94
1,2000-04-08,99
2,2000-09-02,91
2,2000-09-09,87
